# Ideas

- random forests: are there frequent subtrees in the random forests generated from some data set
- labels of vertices in the forests can be of two variants:
    - only the feature on which the split happens
    - feature < threshold
- should we keep the leaf vertices (the decisions) or should we skip them?
- which subtrees are interesting?
    - each vertex in the tree defines a subtree of all vertices below it
    - rooted subtrees in the classical sense
    - unrooted subtrees
    


## prototyping parser

In [38]:
import json
import sys

f = open('forests/adult/text/RF_5.json')
j = json.load(f)
f.close()


In [27]:
print(json.dumps(j, indent=4, sort_keys=True))

[
    {
        "feature": 0,
        "id": 0,
        "isCategorical": "False",
        "leftChild": {
            "feature": 61,
            "id": 1,
            "isCategorical": "False",
            "leftChild": {
                "feature": 50,
                "id": 2,
                "isCategorical": "False",
                "leftChild": {
                    "feature": 60,
                    "id": 3,
                    "isCategorical": "False",
                    "leftChild": {
                        "feature": 0,
                        "id": 4,
                        "isCategorical": "False",
                        "leftChild": {
                            "id": 5,
                            "numSamples": 1453,
                            "prediction": [
                                0.9942806863176419,
                                0.005719313682358117
                            ]
                        },
                        "numSamples": 1869,
              

In [39]:
def recParse(vertex, vertexLabels, edges):
    if 'feature' in vertex.keys():
        vertexLabels[vertex['id'] + 1] = vertex['feature']
        if 'leftChild' in vertex.keys() and 'feature' in vertex['leftChild'].keys():
            edges.append((str(vertex['id'] + 1), str(vertex['leftChild']['id'] + 1), 'leftChild'))
            recParse(vertex['leftChild'], vertexLabels, edges)
        if 'rightChild' in vertex.keys() and 'feature' in vertex['rightChild'].keys():
            edges.append((str(vertex['id'] + 1), str(vertex['rightChild']['id'] + 1), 'rightChild'))
            recParse(vertex['rightChild'], vertexLabels, edges)
            
    return (vertexLabels, edges)

def parseTree(tree):
    vertexLabels = dict()
    edges = list()
    return recParse(tree, vertexLabels, edges)

def transform2GraphDB(vertexLabels, edges, out):
    print(' '.join(['# 0 0', str(len(vertexLabels)), str(len(edges))]))
    for v in sorted(vertexLabels.keys()):
        out.write(str(vertexLabels[v]) + ' ')
    out.write('\n')
        
    for e in edges:
        out.write(' '.join(e) + ' ')
    out.write('\n')
        
for tree in j:
    vertexLabels, edges = parseTree(tree)
#     print(vertexLabels)
#     print(edges)
    transform2GraphDB(vertexLabels, edges, sys.stdout)
print('$')

# 0 0 29 28
0 26 61 51 62 27 53 27 41 0 61 59 51 9 37 63 63 61 23 27 39 43 27 13 60 30 43 20 61 
1 2 leftChild 2 3 leftChild 3 4 leftChild 4 5 leftChild 4 8 rightChild 3 11 rightChild 11 12 leftChild 11 15 rightChild 2 18 rightChild 18 19 leftChild 19 20 leftChild 18 24 rightChild 24 25 leftChild 24 28 rightChild 1 31 rightChild 31 32 leftChild 32 33 leftChild 33 34 leftChild 32 38 rightChild 38 39 leftChild 38 42 rightChild 31 45 rightChild 45 46 leftChild 46 47 leftChild 46 50 rightChild 45 53 rightChild 53 54 leftChild 53 57 rightChild 
# 0 0 27 26
63 61 62 27 10 36 62 0 26 50 53 6 43 13 0 0 18 26 27 48 38 62 9 2 61 3 41 
1 2 leftChild 2 3 leftChild 3 4 leftChild 4 5 leftChild 4 8 rightChild 3 11 rightChild 11 12 leftChild 11 15 rightChild 2 18 rightChild 18 19 leftChild 19 20 leftChild 19 23 rightChild 18 26 rightChild 26 27 leftChild 26 30 rightChild 1 33 rightChild 33 34 leftChild 34 35 leftChild 35 36 leftChild 35 39 rightChild 34 42 rightChild 42 43 leftChild 42 46 rightChild 3

In [22]:
di = dict()
di.keys